# wprowadzenie i GIL

## Wątki

pakiet threading - do programowania wielowątkowego


Uruchomienie funkcji w nowym wątku jest proste:

In [15]:
from random import randint
from time import sleep
from threading import Thread

# Thread(group=None, target=None, name=None, args=(), kwargs={})
# group - zarezerwowane na przyszłość
# target - funkcja do wywołania
# name - unikalna nazwa wątku
# args, kwargs - argumenty przekazywane do funkcji


def f(*args, **kwargs):
    print "f({} ,{})".format(args, kwargs)
    sleep(randint(0, 3))
    return


for i in range(5):
    t = Thread(target=f, args=(i,), kwargs={'kwa': 'rg'})
    t.start()  # uruchomienie wątku 
    #t.join()  # oczekiwanie przez wątek wywołujący na wykonanie wątku wywoływanego
    
print "asdf"

f((0,) ,{'kwa': 'rg'})f((1,) ,{'kwa': 'rg'}) f((2,) ,{'kwa': 'rg'})


f((3,) ,{'kwa': 'rg'})
asdff((4,) ,{'kwa': 'rg'})

